In [201]:
library(tidyverse)
library(lubridate)

In [202]:
df<-read_csv('df.csv',col_names = TRUE,cols(
  Ano = col_double(),
  Mes = col_double(),
  Ano_Mes = col_character(),
  Fecha = col_date(format = ""),
  Precios_Mensuales = col_double(),
  Aportes_Energia_GWh = col_double(),
  Volumen_Util_Diario_Energia_GWh = col_double(),
  Demanda_Energia_SIN_GWh = col_double(),
  ONI = col_double(),
  CEN_Total = col_double(),
  CEN_Hidro = col_double(),
  CEN_Termica = col_double()
))

In [203]:
str(df)

Classes ‘spec_tbl_df’, ‘tbl_df’, ‘tbl’ and 'data.frame':	257 obs. of  12 variables:
 $ Ano                            : num  2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 ...
 $ Mes                            : num  1 2 3 4 5 6 7 8 9 10 ...
 $ Ano_Mes                        : chr  "2000-01" "2000-02" "2000-03" "2000-04" ...
 $ Fecha                          : Date, format: "2000-01-01" "2000-02-01" ...
 $ Precios_Mensuales              : num  36.8 40.3 37.6 44.3 37.3 ...
 $ Aportes_Energia_GWh            : num  1901 1994 2253 2430 4818 ...
 $ Volumen_Util_Diario_Energia_GWh: num  10116 9705 9223 8946 10131 ...
 $ Demanda_Energia_SIN_GWh        : num  3354 3278 3495 3298 3502 ...
 $ ONI                            : num  -1.7 -1.4 -1.1 -0.8 -0.7 -0.6 -0.6 -0.5 -0.5 -0.6 ...
 $ CEN_Total                      : num  11.9 11.9 12 12.1 12.3 ...
 $ CEN_Hidro                      : num  7.97 7.97 8.06 8.14 8.23 ...
 $ CEN_Termica                    : num  3.9 3.9 3.9 3.9 4.05 ...
 - attr(*

# Creacion de funciones

In [204]:
oni<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(ONI)%>%nrow
    df$ONI[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,-0.5,0.5),1)
    
    return(df)
}

In [205]:
aportes<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Aportes_Energia_GWh)%>%nrow
    df$Aportes_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Aportes_Energia_GWh,na.rm = TRUE),max(df$Aportes_Energia_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [206]:
reservas<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Volumen_Util_Diario_Energia_GWh)%>%nrow
    df$Volumen_Util_Diario_Energia_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),max(df$Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [207]:
precios<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Precios_Mensuales)%>%nrow
    df$Precios_Mensuales[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Precios_Mensuales,na.rm = TRUE),max(df$Precios_Mensuales,na.rm = TRUE)),3)
    
    return(df)
}

In [208]:
demanda<-function(df, start_at){
    
    fechaini<-as.Date(start_at)
    n_aleatorios<-df%>%filter(Fecha>=start_at)%>%select(Demanda_Energia_SIN_GWh)%>%nrow
    df$Demanda_Energia_SIN_GWh[df$Fecha>=as.Date(start_at)]<-round(runif(n_aleatorios,min(df$Demanda_Energia_SIN_GWh,na.rm = TRUE),max(df$Demanda_Energia_SIN_GWh,na.rm = TRUE)),3)
    
    return(df)
}

In [209]:
columnas_calculadas<-function(df){
    #debo montar dem_CEN
    df$Dem_CEN<-round(df$Demanda_Energia_SIN_GWh/(as.vector(days_in_month(df$Fecha))*df$CEN_Total*24),4)
    #debo montar Term_Hidro
    df$Termica_Hidraulica<-round(df$CEN_Termica/df$CEN_Hidro,4)
    
    return(df)
}

In [210]:
normalizar_hidrologia<-function(df){
    df<-df%>%
        left_join(
                df%>%arrange(Mes)%>%group_by(Mes)%>%summarize(mean_aportes = mean(Aportes_Energia_GWh,na.rm = TRUE), 
                                                              mean_reservas = mean(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE),
                                                              sd_aportes = sd(Aportes_Energia_GWh, na.rm = TRUE),
                                                              sd_reservas = sd(Volumen_Util_Diario_Energia_GWh,na.rm = TRUE)),by='Mes')%>%
    mutate(
        Aportes_Normalizados = (Aportes_Energia_GWh-mean_aportes)/sd_aportes,
        Reservas_Normalizadas = (Volumen_Util_Diario_Energia_GWh-mean_reservas)/sd_reservas)
    
    return(df)
}

In [211]:
head(df)

Ano,Mes,Ano_Mes,Fecha,Precios_Mensuales,Aportes_Energia_GWh,Volumen_Util_Diario_Energia_GWh,Demanda_Energia_SIN_GWh,ONI,CEN_Total,CEN_Hidro,CEN_Termica
2000,1,2000-01,2000-01-01,36.77878,1900.660,10115.866,3354.237,-1.7,11.88597,7.97497,3.897
2000,2,2000-02,2000-02-01,40.26148,1993.636,9705.065,3278.267,-1.4,11.88597,7.97497,3.897
2000,3,2000-03,2000-03-01,37.63730,2253.361,9222.966,3495.203,-1.1,11.96897,8.05797,3.897
2000,4,2000-04,2000-04-01,44.29806,2429.651,8945.684,3298.386,-0.8,12.05397,8.14297,3.897
2000,5,2000-05,2000-05-01,37.29112,4817.901,10131.148,3502.127,-0.7,12.29927,8.23427,4.051
2000,6,2000-06,2000-06-01,39.99320,4889.563,11020.110,3367.886,-0.6,12.44327,8.23427,4.195


In [212]:
#df%>%arrange(Mes)%>%group_by(Mes)%>%summarize(mean_aportes = mean(Aportes_Energia_GWh), mean_reservas = mean(Volumen_Util_Diario_Energia_GWh),sd_aportes = sd(Aportes_Energia_GWh),sd_reservas = sd(Volumen_Util_Diario_Energia_GWh))
# df<-df%>%
#     left_join(
#             df%>%arrange(Mes)%>%group_by(Mes)%>%summarize(mean_aportes = mean(Aportes_Energia_GWh), 
#                                                           mean_reservas = mean(Volumen_Util_Diario_Energia_GWh),
#                                                           sd_aportes = sd(Aportes_Energia_GWh),
#                                                           sd_reservas = sd(Volumen_Util_Diario_Energia_GWh)),by='Mes')%>%
# mutate(
#     Aportes_Normalizados = (Aportes_Energia_GWh-mean_aportes)/sd_aportes,
#     Reservas_Normalizadas = (Volumen_Util_Diario_Energia_GWh-mean_reservas)/sd_reservas)

# Probar funciones

In [213]:
df<-precios(df,start_at = '2019-06-01')
df<-oni(df,start_at = '2019-06-01')
df<-aportes(df,start_at = '2019-06-01')
df<-reservas(df,start_at = '2019-06-01')
df<-demanda(df,start_at = '2019-06-01')
df<-columnas_calculadas(df)
df<-normalizar_hidrologia(df)
str(df)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	257 obs. of  20 variables:
 $ Ano                            : num  2000 2000 2000 2000 2000 2000 2000 2000 2000 2000 ...
 $ Mes                            : num  1 2 3 4 5 6 7 8 9 10 ...
 $ Ano_Mes                        : chr  "2000-01" "2000-02" "2000-03" "2000-04" ...
 $ Fecha                          : Date, format: "2000-01-01" "2000-02-01" ...
 $ Precios_Mensuales              : num  36.8 40.3 37.6 44.3 37.3 ...
 $ Aportes_Energia_GWh            : num  1901 1994 2253 2430 4818 ...
 $ Volumen_Util_Diario_Energia_GWh: num  10116 9705 9223 8946 10131 ...
 $ Demanda_Energia_SIN_GWh        : num  3354 3278 3495 3298 3502 ...
 $ ONI                            : num  -1.7 -1.4 -1.1 -0.8 -0.7 -0.6 -0.6 -0.5 -0.5 -0.6 ...
 $ CEN_Total                      : num  11.9 11.9 12 12.1 12.3 ...
 $ CEN_Hidro                      : num  7.97 7.97 8.06 8.14 8.23 ...
 $ CEN_Termica                    : num  3.9 3.9 3.9 3.9 4.05 ...
 $ Dem_CEN              

In [215]:
tail(df)

Ano,Mes,Ano_Mes,Fecha,Precios_Mensuales,Aportes_Energia_GWh,Volumen_Util_Diario_Energia_GWh,Demanda_Energia_SIN_GWh,ONI,CEN_Total,CEN_Hidro,CEN_Termica,Dem_CEN,Termica_Hidraulica,mean_aportes,mean_reservas,sd_aportes,sd_reservas,Aportes_Normalizados,Reservas_Normalizadas
2020,12,2020-12,2020-12-01,355.625,1422.426,10968.126,4097.101,0.0,13.15767,8.67967,4.464,0.4185,0.5143,3633.971,11767.435,1505.2603,921.1445,-1.4692107,-0.8677345
2021,1,2021-01,2021-01-01,683.907,1789.253,11259.249,4560.358,0.0,13.15767,8.67967,4.464,0.4659,0.5143,2386.977,10389.987,838.8306,1037.1709,-0.7125680,0.8381090
2021,2,2021-02,2021-02-01,43.783,2724.507,10328.986,3679.246,0.2,13.15867,8.68067,4.464,0.4161,0.5142,2154.000,9093.244,870.8353,1048.3988,0.6551259,1.1786950
2021,3,2021-03,2021-03-01,651.253,7031.393,6917.495,3917.316,-0.3,13.15867,8.68067,4.464,0.4001,0.5142,3088.159,8092.164,1374.5478,1406.5347,2.8687500,-0.8351509
2021,4,2021-04,2021-04-01,196.528,3413.229,11348.939,3477.515,-0.1,13.15867,8.68067,4.464,0.3670,0.5142,4629.707,8383.997,2043.8155,1740.7662,-0.5951994,1.7032394
2021,5,2021-05,2021-05-01,781.577,7852.827,7155.444,5609.168,-0.1,13.16023,8.68223,4.464,0.5729,0.5142,6105.145,9585.005,1696.6865,1903.9624,1.0300557,-1.2760552


In [216]:
generar_archivos<-function(df, start_at, n_iter, folder_location){
    
    
    if(dir.exists('./Resultados/')){
        while(file.remove(list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1])==TRUE){
            list.files(path = folder_location,pattern = '[0-9][0-9][0-9][0-9][0-9].csv',full.names = TRUE)[1]
        }

        fechaini<-as.Date(start_at)

        for( i in seq(n_iter)){

            #Normalizar aportes
            #normalizar reservas
            #Agregar variables calculadas
            df<-precios(df,fechaini)
            df<-oni(df,fechaini)
            df<-aportes(df,fechaini)
            df<-reservas(df,fechaini)
            df<-demanda(df,fechaini)
            df<-columnas_calculadas(df)

            #pronostico de variables calculadas
            write_csv(df, paste0(folder_location,str_pad(i, pad = 0, width = 5),'.csv'))
        }
    } else{
        cat('Error: Directorio no existe en esta mquina')
    }
}

In [217]:
generar_archivos(df,start_at = '2019-06-01',n_iter = 20,folder_location = './Resultados/')

# Tareas

In [218]:
#para el oni es un AR qe estime el P optimo y genere hacia adelante una serie sintetica y llenar 2h
#hacer el arx pronosticar uno a uno y tiene como entrada el ONI y se devuelven las unidades 2h
#hacer arx con oni, aportes, relacion T/H y me devuelvo 2h
#un modelo simple de precio 2h